# 5. Prédire les maladies cardiaques

Nous travaillons sur un jeu de données fourni par le projet __[drivendata](https://www.drivendata.org)__ : __[Warm Up: Machine Learning with a Heart](https://www.drivendata.org/competitions/54/machine-learning-with-a-heart/)__

In [ ]:
import pandas as pd

df = pd.read_csv('/data/heart_values.csv')
df_label = pd.read_csv('/data/heart_labels.csv')
df = pd.merge(df, df_label, on='patient_id')

## 5.1 Analyse exploratoire des données

Explorez ce dataset.

## 5.2 Classement des variables en fonction de leurs types

Ce dataset contient des variables de plusieurs types (numériques, catégorielles et ordinales). Nous devons appliquer des pre-traitements différents pour chaque type de variable.

Identifiez les types pour chaque variable et construisez trois listes Python contenant le nom des variables pour trois types : `var_names_num` (type numérique) `var_names_cat` (type catégoriel), `var_names_ord` (type ordinal). La variable `age` sera traitée différement (i.e. : ne l'incluez dans aucune de ces listes) :

Les variables numériques seront standardisées, mais la classe `StandardScaler` ne peut travailler sur des nombres à virgule flottante. Transformer chacune des variables numérique de type `int64` en `float64` à l'aide de la méthode [pandas.DataFrame.astype](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html) :

## 5.3 Construction du Jeu d'entraînement et du jeu de test

Construisez un jeu d'entrainement et un jeu de test contenant 20% de notre dataset, pensez bien à séparer votre cible `heart_disease_present`. Pour la suite de ce notebook, vos variables devraient être nommées `X_train`, `y_train`, `X_test` et `y_test` :

## 5.4 Transformation des variables

Nous allons créer une chaîne de transformation pour préparer nos données à l'aide de la classe [`sklearn.pipeline.Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

Commencons par les features numériques : créez une instance de `Pipeline` en lui passant comme argument une liste composée d'un tuple de deux valeurs dont la première est un nom simple à retenir (par exemple : `std_scaler`) et la deuxième une instance de [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Nommez cette instance `num_pipeline` :

Faites de même pour :
* les features ordinales avec [`sklearn.preprocessing.OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html), nom de l'étape du Pipeline : `ord_encoder`, nom de l'instance `ord_pipeline`
* les features catégorielles avec [`sklearn.preprocessing.OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) (passez la valeur `'ignore'` au paramètre `handle_unknown`), nom de l'étape du Pipeline : `1h_encoder`, nom de l'instance `cat_pipeline`

Créez un pipeline pour la variable `age`, que vous nommerez `bin_discretizer` et que vous stockerez dans une variable `age_pipeline`, avec [`sklearn.preprocessing.KBinsDiscretizer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html). Vous pouvez passer en paramètres `n_bins=6, strategy='uniform', encode='ordinal'` :

Pour finir nous allons composer nos Pipeline en indiquant sur quelles variables ils s'appliquent.

Créez une instance de [`sklearn.compose.ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) en lui passant en paramètre une liste de tuples de trois valeurs. La première sera un nom simple à retenir (`num`, `ord`, `cat` et `age` par exemple), la deuxième votre variable contenant le pipeline correspondant et la dernière la liste des colonnes sur lesquels ce pipeline doit s'appliquer :

Visualisez l'effet de cette étape de transformation en l'appliquant sur `X_train` par exemple avec la méthode `fit_transform` (vous obtenez un tableau numpy, vous pouvez donc sélectionner la première ligne pour simplifier l'affichage) :

## 5.5 Apprentissage d'une régression logistique

Créez un Pipeline comprenant deux étapes :
* vos pré-traitement, nommé `preparation`
* une instance de [`sklearn.linear_model.LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), nommé `model` avec les paramètres `solver='liblinear', penalty='l2'` :

Créez une grille de paramètre pour ce pipeline où vous ferez varier le paramètre `C` de la régression logistique (`np.logspace(-5, 5, 10)` par exemple), et le paramètre `n_bins` de la binarisation de l'age (`[3,6,12]` par exemple). Les paramètres dans un pipeline sont accessible en précisant le nom de chaque sous-partie du pipeline séparé par deux underscores. Par exemple, `preparation__age__bin_discretizer__n_bins` permet de préciser que nous souhaitons faire varier le paramètre `n_bins` de `bin_discretizer`, lui même étant dans `age`, lui même dans `preparation` (d'où l'importance de bien nommé ses différentes étapes de pipeline) :

Appliquez un grid search sur votre espace de recherche en utilisant la métrique de performance `f1` (paramètre `scoring` de `GridSearchCV` :

Affichez les meilleurs paramètres et le score correspondant :

Calculez les scores avec une validation croisée pour le meilleur modèle sur le jeu d'entrainement :

## 5.6 Apprentissage d'un forêt aléatoire

Faites de même avec l'estimateur [sklearn.ensemble.RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) (exemples pour le grid search : `max_depth` : `np.linspace(10,50,5, dtype=int)`, `min_samples_leaf` : `[2,4,8]`). Quel est le meilleur modèle ?

## 5.7 Analyse des résultats pour le meilleur modèle

Entrainer votre meilleur modèle sur tout le dataset d'entrainement, puis prédisez les classes pour votre dataset de test. Ensuite appliquez les fonctions `precision_score, recall_score, f1_score` du module [`sklearn.metrics`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics). Quel type d'erreurs fait votre classifieur ?

Affichez la matrice de confusion avec [`sklearn.metrics.confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html). Vous pouvez obtenir un graphique de cette matrice avec [`sklearn.metrics.plot_confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html) ou [`sklearn.metrics.ConfusionMatrixDisplay.from_predictions`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html#sklearn.metrics.ConfusionMatrixDisplay.from_predictions) :

## 5.8 Arbre de décision

Le meilleur modèle obtenu est difficilement compréhensible (par des experts métier par exemple). Nous pouvons essayer d'entrainer un modèle plus simple et interprétable : un arbre de décision.

Entrainez un arbre de décision sur les seules variables `max_heart_rate_achieved` et `oldpeak_eq_st_depression` avec une profondeur maximale de deux. Calculez la performance du modèle obtenu. Puis afficher cet arbre avec [`sklearn.tree.export_graphviz`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html). Vous pouvez enregistrer la sortie dans un fichier `.dot` et afficher le résultats avec le transformer en image avec le service : [webgraphviz.com](http://webgraphviz.com/)

Essayez d'améliorer le score obtenu avec un arbre de décision tout en conservant un modèle interprétable (features utilisées et profondeur maximale).